In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/Bigcontest

/content/drive/MyDrive/Colab Notebooks/Bigcontest


앱 사용성 데이터를 통한 대출신청 예측분석
- 가명화된 데이터를 기반으로 고객의 대출상품 신청여부 예측
(2022년 3~5월 데이터제공 / 2022년 6월 예측)
- 예측모델을 활용하여 탐색적 데이터 분석 수행
- 대출신청, 미신청 고객을 분류하여 고객의 특성 분석결과 도출

In [3]:
import pandas as pd
import os

In [4]:
user = pd.read_csv('./data/user_spec.csv') # 유저 스펙 테이블(신용정보)
log = pd.read_csv('./data/log_data.csv') # 사용자가 신청한 대출/금융사별 승인결과 (타겟)
loan = pd.read_csv('./data/loan_result.csv') # finda 로그 정보

# user_spec
상대적으로 신용점수는 높고 수입은 적은 경우, 신용점수는 낮고 수입은 많은 경우가 있는 것 같음. 시간, 분, 초 단위

# log_data
user의 정보 중에서 event(행동명)은 순서가 있을 것으로 예상되나, 비슷한 시간에 같은 event가 벌어진 로그가 존재하는 것으로 보임. timestamp는 초 단위.

# loan_result
동시에 금융사 별로 다양한 상품을 조회해서 승인 결과를 반환. 여기는 시간, 분 단위

In [5]:
user.shape

(1394216, 17)

In [6]:
log.shape

(17843993, 6)

In [7]:
loan.shape

(13527363, 7)

In [8]:
user.head()

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,기타가족소유,30000000.0,대환대출,0.0,NaN,1.0,27000000.0
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,기타가족소유,10000000.0,생활비,0.0,NaN,5.0,15000000.0
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,자가,2000000.0,생활비,0.0,NaN,7.0,344000000.0
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,1.0,16000000.0


In [9]:
user['birth_year'].describe()

count    1.381255e+06
mean     1.983285e+03
std      1.067367e+01
min      1.927000e+03
25%      1.976000e+03
50%      1.984000e+03
75%      1.992000e+03
max      2.008000e+03
Name: birth_year, dtype: float64

In [10]:
minor = user.loc[user['birth_year'] > 2004]

In [11]:
minor['birth_year'].sort_values(ascending=False)

638067     2008.0
649038     2008.0
605500     2008.0
1245518    2008.0
213253     2008.0
270860     2008.0
1164717    2008.0
725651     2007.0
744880     2007.0
1075524    2007.0
465474     2007.0
447432     2007.0
426583     2007.0
674849     2007.0
267247     2007.0
157790     2007.0
400475     2007.0
1220100    2006.0
1135456    2006.0
1220567    2006.0
1124351    2006.0
1112780    2006.0
1230783    2006.0
787573     2006.0
1231832    2006.0
111609     2006.0
1325718    2006.0
1379923    2006.0
505350     2006.0
479147     2006.0
544431     2006.0
122044     2006.0
154434     2006.0
253265     2006.0
555270     2006.0
1343326    2005.0
1275895    2005.0
142639     2005.0
186027     2005.0
354668     2005.0
1210809    2005.0
377474     2005.0
429919     2005.0
591656     2005.0
1115694    2005.0
1097877    2005.0
1088701    2005.0
520318     2005.0
1070055    2005.0
1044616    2005.0
1013403    2005.0
882868     2005.0
815749     2005.0
804923     2005.0
520627     2005.0
768399    

In [12]:
len(minor)

57

개인회생 신청자
1. 가장 어린 개인회생자는 2007년생
  - 대출한 이력이 있는데 기대출금액이 NaN..?
  - 왜.. 숫자.. 아냐..?
  - 미성년자는 57명
  - 대출 승인 여부: 
2. 가장 나이 많은 개인회생자는 1938년생
  - 연수입 1500만원, 기대출금액 100만원
  - 대출 승인 여부: 
3. 최저 신용점수는 100, 2606명
  - 세세히 뜯어볼 것.
4. 개인회생자이나 신용점수가 1000점, 6명 있음
5. 연수입 0 ~ 9900000000
6. 2022년 6월 30일 입사자도 존재
7. 대출 희망 금액이 0인 경우 존재
8. 기대출수는 1~59
9. 기대출금액은 0~1064000000
10. existing_loan_amt 왜 자꾸 NaN
11. 개인회생 납입 완료가 되지 않았는데 기대출금액이 NaN?
12. 개인회생자였거나 개인회생자인 사람이 대출 가능한가? 확인해봐야함

NaN의 의미가, 개인회생으로 인해서 갚을 필요가 없어져서?
그렇다면 개인회생자가 아닌 경우에도 NaN이 있나 확인해보고,
NaN 결측치를 채울 때 임의에 따라 구분해볼 수 있겠다.

birth_year와 gender가 존재하지 않는 사람?
-> 로그데이터랑 비교해서 찾아봐야 함
-> 회원가입이 완료되지 않았거나(비정상종료?)

신용점수가 없는 사람들 존재
1. 미성년자일 가능성

In [13]:
rehab = user.loc[user['personal_rehabilitation_yn'] == 1]
rehab.describe()

,application_id,user_id,birth_year,gender,credit_score,yearly_income,company_enter_month,desired_amount,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
count,1.270900e+04,12709.000000,12501.000000,12501.000000,11340.000000,1.270900e+04,1.052500e+04,1.270900e+04,12709.0,12709.000000,10209.000000,8.537000e+03
mean,1.089808e+06,444180.942796,1981.624030,0.659147,449.203704,4.079503e+07,8.715767e+06,3.191534e+07,1.0,0.106067,2.668332,3.372344e+07
std,6.261633e+05,254630.837898,10.517402,0.474015,236.976038,1.725586e+08,9.884504e+06,3.118970e+08,0.0,0.307935,2.518719,4.143254e+07
min,8.000000e+01,24.000000,1938.000000,0.000000,100.000000,0.000000e+00,1.970070e+05,0.000000e+00,1.0,0.000000,1.000000,0.000000e+00
25%,5.510940e+05,220116.000000,1975.000000,0.000000,150.000000,2.300000e+07,2.020090e+05,3.000000e+06,1.0,0.000000,1.000000,1.000000e+07
50%,1.092478e+06,451567.000000,1982.000000,1.000000,540.000000,3.000000e+07,2.022020e+05,6.000000e+06,1.0,0.000000,2.000000,2.300000e+07
75%,1.635770e+06,664234.000000,1990.000000,1.000000,620.000000,4.000000e+07,2.020090e+07,1.500000e+07,1.0,0.000000,3.000000,4.300000e+07
max,2.167754e+06,879670.000000,2007.000000,1.000000,1000.000000,9.900000e+09,2.022063e+07,1.000000e+10,1.0,1.000000,59.000000,1.064000e+09


In [14]:
#가장 어린 개인회생자
rehab = user.loc[(user['personal_rehabilitation_yn'] == 1) & (user['birth_year'] == 2007)]
rehab

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
447432,1482913,671192,2007.0,1.0,2022-05-26 12:17:52,NaN,2000000.0,OTHERINCOME,NaN,기타,전월세,1000000.0,생활비,1.0,1.0,1.0,NaN


In [15]:
#가장 나이 많은 개인회생자
rehab = user.loc[(user['personal_rehabilitation_yn'] == 1) & (user['birth_year'] == 1938)]
rehab

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
269429,523808,661566,1938.0,1.0,2022-05-04 13:53:34,650.0,15000000.0,OTHERINCOME,NaN,기타,전월세,12000000.0,생활비,1.0,0.0,1.0,1000000.0


In [16]:
# 신용점수가 100점인 개인회생자
rehab = user.loc[(user['personal_rehabilitation_yn'] == 1) & (user['credit_score'] == 100)]
print(len(rehab))
rehab

2606


,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
170,1640338,298318,1976.0,1.0,2022-04-20 08:16:04,100.0,50000000.0,EARNEDINCOME,201406.0,정규직,전월세,5000000.0,생활비,1.0,0.0,1.0,25000000.0
1127,1917588,381184,1981.0,1.0,2022-05-06 16:06:46,100.0,71000000.0,FREELANCER,202107.0,기타,전월세,5000000.0,생활비,1.0,0.0,1.0,15000000.0
2544,1472383,476741,1984.0,1.0,2022-06-10 02:26:23,100.0,64000000.0,EARNEDINCOME,20080901.0,정규직,전월세,5000000.0,생활비,1.0,0.0,6.0,99000000.0
3220,941325,515547,1983.0,0.0,2022-06-07 09:17:37,100.0,28000000.0,EARNEDINCOME,20210501.0,정규직,전월세,50000000.0,생활비,1.0,0.0,3.0,45000000.0
3295,1790745,442063,1980.0,1.0,2022-04-20 10:10:41,100.0,22000000.0,EARNEDINCOME,202102.0,정규직,기타가족소유,3000000.0,생활비,1.0,0.0,2.0,37000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392059,1723523,11052,1990.0,0.0,2022-05-25 23:37:28,100.0,24000000.0,EARNEDINCOME2,202010.0,정규직,자가,16000000.0,대환대출,1.0,1.0,1.0,25000000.0
1392073,272266,717275,1982.0,0.0,2022-05-25 12:13:07,100.0,50000000.0,OTHERINCOME,NaN,기타,전월세,3000000.0,생활비,1.0,0.0,1.0,4000000.0
1392111,1754309,558701,1975.0,1.0,2022-05-15 16:42:52,100.0,35000000.0,EARNEDINCOME,201903.0,정규직,전월세,2000000.0,생활비,1.0,0.0,3.0,45000000.0
1393103,1703638,414806,1972.0,1.0,2022-05-29 13:04:02,100.0,35000000.0,OTHERINCOME,NaN,기타,기타가족소유,2000000.0,생활비,1.0,0.0,1.0,NaN


In [17]:
# 신용점수가 1000점인 개인회생자
rehab = user.loc[(user['personal_rehabilitation_yn'] == 1) & (user['credit_score'] == 1000)]
rehab

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
101328,260128,668927,1973.0,1.0,2022-05-13 02:15:31,1000.0,65000000.0,OTHERINCOME,NaN,기타,기타가족소유,99000000.0,주택구입,1.0,1.0,1.0,NaN
501497,903568,163595,1969.0,1.0,2022-06-29 15:53:45,1000.0,0.0,FREELANCER,20220629.0,기타,전월세,50000000.0,생활비,1.0,0.0,10.0,276000000.0
561681,286793,322976,1988.0,0.0,2022-06-29 15:17:09,1000.0,50000000.0,PRIVATEBUSINESS,20200629.0,기타,전월세,50000000.0,생활비,1.0,0.0,2.0,476000000.0
585674,1985522,696885,1979.0,0.0,2022-04-26 13:26:08,1000.0,55000000.0,EARNEDINCOME,201311.0,정규직,자가,30000000.0,주택구입,1.0,0.0,2.0,24000000.0
932305,1041173,363512,1983.0,0.0,2022-05-10 14:59:25,1000.0,50000000.0,EARNEDINCOME,200511.0,계약직,자가,100000000.0,투자,1.0,0.0,1.0,0.0
1305207,1217636,24913,1965.0,1.0,2022-05-09 12:18:02,1000.0,40000000.0,OTHERINCOME,NaN,기타,전월세,200000000.0,전월세보증금,1.0,0.0,NaN,NaN


In [18]:
# 연수입 0
rehab = user.loc[(user['personal_rehabilitation_yn'] == 1) & (user['yearly_income'] == 0)]
rehab

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
1677,713972,202009,1993.0,0.0,2022-06-14 21:46:14,NaN,0.0,EARNEDINCOME,20220501.0,정규직,기타가족소유,3000000.0,생활비,1.0,0.0,NaN,NaN
12931,503359,582175,1988.0,0.0,2022-06-24 22:18:22,NaN,0.0,OTHERINCOME,NaN,기타,기타가족소유,3000000.0,대환대출,1.0,0.0,NaN,NaN
15487,1515127,248086,1993.0,1.0,2022-04-25 19:07:31,700.0,0.0,FREELANCER,202204.0,기타,자가,2000000.0,생활비,1.0,0.0,1.0,NaN
24622,1901869,548941,1971.0,0.0,2022-06-20 14:33:27,640.0,0.0,OTHERINCOME,NaN,기타,기타가족소유,5000000.0,생활비,1.0,0.0,5.0,16000000.0
25690,523549,449174,1980.0,1.0,2022-06-28 14:58:57,250.0,0.0,OTHERINCOME,NaN,기타,전월세,1000000.0,생활비,1.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374460,1257875,512285,2001.0,1.0,2022-05-16 21:44:16,760.0,0.0,OTHERINCOME,NaN,기타,기타가족소유,20000000.0,사업자금,1.0,0.0,2.0,8000000.0
1375471,1348837,549645,1988.0,1.0,2022-06-30 11:50:34,100.0,0.0,FREELANCER,20211130.0,기타,기타가족소유,18000000.0,대환대출,1.0,0.0,1.0,13000000.0
1382113,2148850,345950,1994.0,1.0,2022-06-29 12:00:02,100.0,0.0,OTHERINCOME,NaN,기타,자가,1000000.0,생활비,1.0,0.0,NaN,NaN
1386950,320131,674746,1983.0,0.0,2022-06-27 08:52:31,100.0,0.0,OTHERINCOME,NaN,기타,전월세,5000000.0,생활비,1.0,0.0,4.0,30000000.0


In [19]:
# 연수입 9900000000
rehab = user.loc[(user['personal_rehabilitation_yn'] == 1) & (user['yearly_income'] == 9900000000)]
rehab

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
432739,560269,496416,1978.0,1.0,2022-05-11 21:59:05,700.0,9.900000e+09,FREELANCER,202205.0,기타,전월세,9.989000e+09,생활비,1.0,0.0,1.0,NaN
